# Prepare USM job for model comparison

For USM, one nested setup down to 2 metre resolution is created.


In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from src.config import get_config, get_dask_cluster

config = get_config()
cluster, client = get_dask_cluster(config)

In [ ]:
import xarray as xr
import numpy as np
import scipy.ndimage

from src.job_generation import (
    Job,
    read_namelist,
    Driver,
    JobNest,
    update_dict_recursive,
    apply_buffer_to_usm_driver,
    set_initial_soil_conditions_from_precursor,
    set_surface_pressure_to_dynamic,
)

## Prepare job objects

Unlike with the SLUrb job, the USM job is a hybrid of top-down and bottom-up construction. The detailed building map is downsampled from the original resolution for the coarser jobs, whilst the surrounding vegetation is upsampled from the coarse resolution.


In [ ]:
job_usm = Job("slurb_c_usm")
job_usm.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_coarse_p3d.yml"
)
update_dict_recursive(
    job_usm.p3d,
    read_namelist(Path(config.path.experiments.comparison) / "usm_coarse_p3d.yml"),
)

job_usm.p3dr = job_usm.p3d.copy()
job_usm.p3dr["initialization_parameters"]["initializing_actions"] = "read_restart_data"

### Load surface configuration shared by the cases


In [ ]:
lcz_map = xr.open_dataset(Path(config.path.data.raw) / "lcz" / "lcz_map.nc")["lcz"]
usm_driver = xr.open_dataset(Path(config.path.data.raw) / "lcz" / "USM_lcz_fine.nc")

Apply buffer zones to USM driver and offset coordinates to origin of the coarse domain.


In [ ]:
usm_driver, urban_mask = apply_buffer_to_usm_driver(usm_driver, buffer=32)
usm_driver["x"] = usm_driver["x"] + 2368
usm_driver["y"] = usm_driver["y"] + 1280
urban_mask["x"] = urban_mask["x"] + 2368
urban_mask["y"] = urban_mask["y"] + 1280

To simplify reindexing, use only 2D building map. This doesn't make a difference as there are no overhanging structures. Furthermore, drop all parameter inputs, these will be set using user code interface to exact same values as in SLUrb run.


In [ ]:
usm_driver

In [ ]:
usm_driver = usm_driver.drop_vars(
    ["buildings_3d", "building_pars", "street_crossing", "street_type"]
)
usm_driver = usm_driver.drop_dims(["z", "nbuilding_pars"])
usm_driver["vegetation_pars"][:] = np.nan

Fix some inconsistencies in the input, e.g. non-unique building ids.


In [ ]:
usm_driver["building_id"][:] = scipy.ndimage.label(
    usm_driver["buildings_2d"] > 0.0, output=np.int32
)[0]
usm_driver["building_id"] = usm_driver["building_id"].where(
    usm_driver["building_id"] > 0, other=-9999
)
usm_driver["building_id"] = usm_driver["building_id"].astype(np.int32)
usm_driver["building_id"].attrs["_FillValue"] = -9999
usm_driver["building_id"].attrs["force_dtype"] = 1

In [ ]:
usm_driver["pavement_type"] = usm_driver["pavement_type"].where(
    usm_driver["pavement_type"] != 1, other=2
)
usm_driver["soil_type"] = usm_driver["soil_type"].where(
    ~np.isnan(usm_driver["soil_type"]), other=2
)

In [ ]:
for surf_type in (
    "building_type",
    "pavement_type",
    "soil_type",
    "vegetation_type",
    "water_type",
):
    usm_driver[surf_type] = usm_driver[surf_type].fillna(-127)
    usm_driver[surf_type] = usm_driver[surf_type].astype(np.int8)
    usm_driver[surf_type].attrs["_FillValue"] = -127

Same static input as for SLUrb job is used outside the urban area.


In [ ]:
static_template = xr.open_dataset(
    Path(config.path.data.jobs) / "slurb_c_slurb" / "INPUT" / "slurb_c_slurb_static"
)
for surf_type in ("soil_type", "pavement_type", "vegetation_type", "water_type"):
    static_template[surf_type] = static_template[surf_type].fillna(-127)
    static_template[surf_type] = static_template[surf_type].astype(np.int8)
    static_template[surf_type].attrs["_FillValue"] = -127

## Coarse domain


### Dynamic driver

We reuse the same dynamic driver for the root from the SLUrb setup except for the initial soil temperature and moisture as these will be set only for vegetation patches.


In [ ]:
job_usm.register_driver("dynamic", Driver())
job_usm.drivers["dynamic"].set_grid("uvws", vertical=True)
job_usm.drivers["dynamic"].set_zsoil()
job_usm.drivers["dynamic"].ds = xr.open_dataset(
    Path(config.path.data.jobs) / "slurb_c_slurb" / "INPUT" / "slurb_c_slurb_dynamic",
    chunks={"time": "auto"},
)

# Set initial wind speed BC at urban canopy to zero. As the initial wind profile is from
# the precursor with all-flat topography, the initial wind speed within the urban canopy would
# be unrealistically high.
u_init = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_u"]
    .where(job_usm.drivers["dynamic"].ds["z"] >= 24.0, other=0.0)
)
v_init = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_v"]
    .where(job_usm.drivers["dynamic"].ds["z"] >= 24.0, other=0.0)
)
wspeed_bc = xr.DataArray([0], dims=["z"], coords={"z": [0]})
u_init = xr.concat([wspeed_bc, u_init], dim="z", combine_attrs="no_conflicts").compute()
v_init = xr.concat([wspeed_bc, v_init], dim="z", combine_attrs="no_conflicts").compute()
u_init = u_init.interpolate_na(dim="z", method="linear")
v_init = v_init.interpolate_na(dim="z", method="linear")

job_usm.drivers["dynamic"].ds["init_atmosphere_u"] = u_init.interp(
    z=job_usm.drivers["dynamic"].ds["z"],
    method="linear",
)
job_usm.drivers["dynamic"].ds["init_atmosphere_v"] = v_init.interp(
    z=job_usm.drivers["dynamic"].ds["z"],
    method="linear",
)

### Initial material temperatures

The USM spinup doesn't model the atmosphere within the urban canopy at all, causing deviation at the initialization. To ensure comparability, we initialize the material temperatures based on SLUrb model spinup. These values are copied to user code for initializasion.


In [ ]:
slurb_init_3d = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_c_slurb"
    / "OUTPUT"
    / "slurb_c_slurb_3d.000.nc",
    decode_times=False,
).isel(time=0)

print(
    f"""Road: {slurb_init_3d["slurb_t_road"].mean(dim=("x","y")).round(decimals=4).values}"""
)
print(
    f"""Roofs: {slurb_init_3d["slurb_t_roof"].mean(dim=("x","y")).round(decimals=4).values}"""
)
print(
    f"""Walls: {slurb_init_3d["slurb_t_wall_a"].mean(dim=("x","y")).round(decimals=4).values}"""
)
print(
    f"""Windows: {slurb_init_3d["slurb_t_win_a"].mean(dim=("x","y")).round(decimals=4).values}"""
)

### Static driver

Combine fine-resolution static driver from LCZ generator and the static driver from the SLUrb run.


In [ ]:
job_usm.register_driver("static", Driver())
job_usm.drivers["static"].set_grid("s", vertical=False)
job_usm.drivers["static"].ds = usm_driver.reindex_like(
    job_usm.drivers["static"].ds, method="nearest"
)
for var in static_template.data_vars.keys():
    job_usm.drivers["static"].ds[var] = (
        job_usm.drivers["static"]
        .ds[var]
        .where(
            urban_mask.reindex_like(
                job_usm.drivers["static"].ds[var], method="nearest"
            ),
            other=static_template[var].reindex_like(
                job_usm.drivers["static"].ds[var], method="nearest"
            ),
        )
    )

### Initial soil conditions from precursor


In [ ]:
pre_3d = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_3d.001.nc",
    decode_times=False,
)
pre_static = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)
job_usm.drivers["dynamic"] = set_initial_soil_conditions_from_precursor(
    pre_3d,
    pre_static["vegetation_type"],
    job_usm.drivers["dynamic"],
    job_usm.drivers["static"].ds["vegetation_type"],
)

## Medium-coarse nest


In [ ]:
nest_medium_coarse = JobNest(root=job_usm, nest_id=2)

### Namelists


In [ ]:
nest_medium_coarse.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_medium_coarse_p3d.yml"
)
update_dict_recursive(
    nest_medium_coarse.p3d,
    read_namelist(
        Path(config.path.experiments.comparison) / "usm_medium_coarse_p3d.yml"
    ),
)
nest_medium_coarse.p3dr = nest_medium_coarse.p3d.copy()
nest_medium_coarse.p3dr["initialization_parameters"]["initializing_actions"] = (
    "read_restart_data"
)

### Static driver


In [ ]:
# Refined domain for the medium nest by resampling
nest_medium_coarse.register_driver("static", Driver())
nest_medium_coarse.drivers["static"].set_grid("s", vertical=False)
# Offsetting the coordinates is required for reindex_like
nest_medium_coarse.drivers["static"].ds["x"] = (
    nest_medium_coarse.drivers["static"].ds["x"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["static"].ds["y"] = (
    nest_medium_coarse.drivers["static"].ds["y"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][12]
)
nest_medium_coarse.drivers["static"].ds = usm_driver.reindex_like(
    nest_medium_coarse.drivers["static"].ds, method="nearest"
)
for var in static_template.data_vars.keys():
    nest_medium_coarse.drivers["static"].ds[var] = (
        nest_medium_coarse.drivers["static"]
        .ds[var]
        .where(
            urban_mask.reindex_like(
                nest_medium_coarse.drivers["static"].ds, method="nearest"
            ),
            other=static_template[var].reindex_like(
                nest_medium_coarse.drivers["static"].ds, method="nearest"
            ),
        )
    )
nest_medium_coarse.drivers["static"].ds["x"] = (
    nest_medium_coarse.drivers["static"].ds["x"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["static"].ds["y"] = (
    nest_medium_coarse.drivers["static"].ds["y"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][12]
)

### Dynamic driver


In [ ]:
nest_medium_coarse.register_driver("dynamic", Driver())
nest_medium_coarse.drivers["dynamic"].set_grid("uvws", vertical=True)
nest_medium_coarse.drivers["dynamic"].set_zsoil()

nest_medium_coarse.drivers["dynamic"].ds["x"] = (
    nest_medium_coarse.drivers["dynamic"].ds["x"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["dynamic"].ds["y"] = (
    nest_medium_coarse.drivers["dynamic"].ds["y"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][12]
)
nest_medium_coarse.drivers["dynamic"].ds["init_soil_t"] = (
    job_usm.drivers["dynamic"]
    .ds["init_soil_t"]
    .reindex_like(nest_medium_coarse.drivers["dynamic"].ds, method="nearest")
)
nest_medium_coarse.drivers["dynamic"].ds["init_soil_m"] = (
    job_usm.drivers["dynamic"]
    .ds["init_soil_m"]
    .reindex_like(nest_medium_coarse.drivers["dynamic"].ds, method="nearest")
)
nest_medium_coarse.drivers["dynamic"].ds["x"] = (
    nest_medium_coarse.drivers["dynamic"].ds["x"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["dynamic"].ds["y"] = (
    nest_medium_coarse.drivers["dynamic"].ds["y"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][12]
)

In [ ]:
nest_medium_coarse.drivers["dynamic"].ds["ls_forcing_ug"] = (
    job_usm.drivers["dynamic"]
    .ds["ls_forcing_ug"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["ls_forcing_vg"] = (
    job_usm.drivers["dynamic"]
    .ds["ls_forcing_vg"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)

nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_u"] = u_init.interp(
    z=nest_medium_coarse.drivers["dynamic"].ds["z"],
    method="linear",
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_v"] = v_init.interp(
    z=nest_medium_coarse.drivers["dynamic"].ds["z"],
    method="linear",
)

nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_pt"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_pt"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_qv"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_qv"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_w"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_w"]
    .interp(
        zw=nest_medium_coarse.drivers["dynamic"].ds["zw"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    nest_medium_coarse.drivers["dynamic"], p0=1e5
)

In [ ]:
nest_medium_coarse.drivers["dynamic"].ds["rad_sw_in"] = job_usm.drivers["dynamic"].ds[
    "rad_sw_in"
]
nest_medium_coarse.drivers["dynamic"].ds["rad_lw_in"] = job_usm.drivers["dynamic"].ds[
    "rad_lw_in"
]

In [ ]:
nest_medium_coarse.drivers["dynamic"] = set_initial_soil_conditions_from_precursor(
    pre_3d,
    pre_static["vegetation_type"],
    nest_medium_coarse.drivers["dynamic"],
    nest_medium_coarse.drivers["static"].ds["vegetation_type"],
)

## Medium-fine nest


In [ ]:
nest_medium_fine = JobNest(root=job_usm, nest_id=3)

In [ ]:
nest_medium_fine.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_medium_fine_p3d.yml"
)
update_dict_recursive(
    nest_medium_fine.p3d,
    read_namelist(Path(config.path.experiments.comparison) / "usm_medium_fine_p3d.yml"),
)
nest_medium_fine.p3dr = nest_medium_fine.p3d.copy()
nest_medium_fine.p3dr["initialization_parameters"]["initializing_actions"] = (
    "read_restart_data"
)

### Static driver


In [ ]:
# Refined domain for the medium nest by resampling
nest_medium_fine.register_driver("static", Driver())
nest_medium_fine.drivers["static"].set_grid("s", vertical=False)
# Offsetting the coordinates is required for reindex_like
nest_medium_fine.drivers["static"].ds["x"] = (
    nest_medium_fine.drivers["static"].ds["x"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["static"].ds["y"] = (
    nest_medium_fine.drivers["static"].ds["y"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][19]
)
nest_medium_fine.drivers["static"].ds = usm_driver.reindex_like(
    nest_medium_fine.drivers["static"].ds, method="nearest"
)
for var in static_template.data_vars.keys():
    nest_medium_fine.drivers["static"].ds[var] = (
        nest_medium_fine.drivers["static"]
        .ds[var]
        .where(
            urban_mask.reindex_like(
                nest_medium_fine.drivers["static"].ds, method="nearest"
            ),
            other=static_template[var].reindex_like(
                nest_medium_fine.drivers["static"].ds, method="nearest"
            ),
        )
    )
nest_medium_fine.drivers["static"].ds["x"] = (
    nest_medium_fine.drivers["static"].ds["x"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["static"].ds["y"] = (
    nest_medium_fine.drivers["static"].ds["y"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][19]
)

### Dynamic driver


In [ ]:
nest_medium_fine.register_driver("dynamic", Driver())
nest_medium_fine.drivers["dynamic"].set_grid("uvws", vertical=True)
nest_medium_fine.drivers["dynamic"].set_zsoil()

nest_medium_fine.drivers["dynamic"].ds["x"] = (
    nest_medium_fine.drivers["dynamic"].ds["x"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["dynamic"].ds["y"] = (
    nest_medium_fine.drivers["dynamic"].ds["y"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][19]
)
nest_medium_fine.drivers["dynamic"].ds["init_soil_t"] = (
    job_usm.drivers["dynamic"]
    .ds["init_soil_t"]
    .reindex_like(nest_medium_fine.drivers["dynamic"].ds, method="nearest")
)
nest_medium_fine.drivers["dynamic"].ds["init_soil_m"] = (
    job_usm.drivers["dynamic"]
    .ds["init_soil_m"]
    .reindex_like(nest_medium_fine.drivers["dynamic"].ds, method="nearest")
)
nest_medium_fine.drivers["dynamic"].ds["x"] = (
    nest_medium_fine.drivers["dynamic"].ds["x"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["dynamic"].ds["y"] = (
    nest_medium_fine.drivers["dynamic"].ds["y"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][19]
)

In [ ]:
nest_medium_fine.drivers["dynamic"].ds["ls_forcing_ug"] = (
    job_usm.drivers["dynamic"]
    .ds["ls_forcing_ug"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["ls_forcing_vg"] = (
    job_usm.drivers["dynamic"]
    .ds["ls_forcing_vg"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_u"] = u_init.interp(
    z=nest_medium_fine.drivers["dynamic"].ds["z"],
    method="linear",
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_v"] = v_init.interp(
    z=nest_medium_fine.drivers["dynamic"].ds["z"],
    method="linear",
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_pt"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_pt"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_qv"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_qv"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_w"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_w"]
    .interp(
        zw=nest_medium_fine.drivers["dynamic"].ds["zw"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    nest_medium_fine.drivers["dynamic"], p0=1e5
)

In [ ]:
nest_medium_fine.drivers["dynamic"].ds["rad_sw_in"] = job_usm.drivers["dynamic"].ds[
    "rad_sw_in"
]
nest_medium_fine.drivers["dynamic"].ds["rad_lw_in"] = job_usm.drivers["dynamic"].ds[
    "rad_lw_in"
]

In [ ]:
nest_medium_fine.drivers["dynamic"] = set_initial_soil_conditions_from_precursor(
    pre_3d,
    pre_static["vegetation_type"],
    nest_medium_fine.drivers["dynamic"],
    nest_medium_fine.drivers["static"].ds["vegetation_type"],
)

## Fine nest


In [ ]:
nest_fine = JobNest(root=job_usm, nest_id=4)

In [ ]:
nest_fine.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "usm_fine_p3d.yml"
)
nest_fine.p3dr = nest_fine.p3d.copy()
nest_fine.p3dr["initialization_parameters"]["initializing_actions"] = (
    "read_restart_data"
)

### Static driver


In [ ]:
nest_fine.register_driver("static", Driver())
nest_fine.drivers["static"].set_grid("s", vertical=False)
# Offsetting the coordinates is required for reindex_like
nest_fine.drivers["static"].ds["x"] = (
    nest_fine.drivers["static"].ds["x"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][25]
)
nest_fine.drivers["static"].ds["y"] = (
    nest_fine.drivers["static"].ds["y"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][26]
)
nest_fine.drivers["static"].ds = usm_driver.reindex_like(
    nest_fine.drivers["static"].ds, method="nearest"
)
for var in static_template.data_vars.keys():
    nest_fine.drivers["static"].ds[var] = (
        nest_fine.drivers["static"]
        .ds[var]
        .where(
            urban_mask.reindex_like(nest_fine.drivers["static"].ds, method="nearest"),
            other=static_template[var].reindex_like(
                nest_fine.drivers["static"].ds, method="nearest"
            ),
        )
    )
nest_fine.drivers["static"].ds["x"] = (
    nest_fine.drivers["static"].ds["x"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][25]
)
nest_fine.drivers["static"].ds["y"] = (
    nest_fine.drivers["static"].ds["y"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][26]
)

### Dynamic driver


In [ ]:
nest_fine.register_driver("dynamic", Driver())
nest_fine.drivers["dynamic"].set_grid("uvws", vertical=True)
nest_fine.drivers["dynamic"].set_zsoil()

nest_fine.drivers["dynamic"].ds["x"] = (
    nest_fine.drivers["dynamic"].ds["x"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][25]
)
nest_fine.drivers["dynamic"].ds["y"] = (
    nest_fine.drivers["dynamic"].ds["y"]
    + job_usm.p3d["nesting_parameters"]["domain_layouts"][26]
)
nest_fine.drivers["dynamic"].ds["init_soil_t"] = (
    job_usm.drivers["dynamic"]
    .ds["init_soil_t"]
    .reindex_like(nest_fine.drivers["dynamic"].ds, method="nearest")
)
nest_fine.drivers["dynamic"].ds["init_soil_m"] = (
    job_usm.drivers["dynamic"]
    .ds["init_soil_m"]
    .reindex_like(nest_fine.drivers["dynamic"].ds, method="nearest")
)
nest_fine.drivers["dynamic"].ds["x"] = (
    nest_fine.drivers["dynamic"].ds["x"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][25]
)
nest_fine.drivers["dynamic"].ds["y"] = (
    nest_fine.drivers["dynamic"].ds["y"]
    - job_usm.p3d["nesting_parameters"]["domain_layouts"][26]
)

In [ ]:
nest_fine.drivers["dynamic"].ds["ls_forcing_ug"] = (
    job_usm.drivers["dynamic"]
    .ds["ls_forcing_ug"]
    .interp(
        z=nest_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_fine.drivers["dynamic"].ds["ls_forcing_vg"] = (
    job_usm.drivers["dynamic"]
    .ds["ls_forcing_vg"]
    .interp(
        z=nest_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_fine.drivers["dynamic"].ds["init_atmosphere_u"] = u_init.interp(
    z=nest_fine.drivers["dynamic"].ds["z"],
    method="linear",
)
nest_fine.drivers["dynamic"].ds["init_atmosphere_v"] = v_init.interp(
    z=nest_fine.drivers["dynamic"].ds["z"],
    method="linear",
)
nest_fine.drivers["dynamic"].ds["init_atmosphere_pt"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_pt"]
    .interp(
        z=nest_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_fine.drivers["dynamic"].ds["init_atmosphere_qv"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_qv"]
    .interp(
        z=nest_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_fine.drivers["dynamic"].ds["init_atmosphere_w"] = (
    job_usm.drivers["dynamic"]
    .ds["init_atmosphere_w"]
    .interp(
        zw=nest_fine.drivers["dynamic"].ds["zw"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_fine.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    nest_fine.drivers["dynamic"], p0=1e5
)

In [ ]:
nest_fine.drivers["dynamic"].ds["rad_sw_in"] = job_usm.drivers["dynamic"].ds[
    "rad_sw_in"
]
nest_fine.drivers["dynamic"].ds["rad_lw_in"] = job_usm.drivers["dynamic"].ds[
    "rad_lw_in"
]

In [ ]:
nest_fine.drivers["dynamic"] = set_initial_soil_conditions_from_precursor(
    pre_3d,
    pre_static["vegetation_type"],
    nest_fine.drivers["dynamic"],
    nest_fine.drivers["static"].ds["vegetation_type"],
)

## Write job files to storage


In [ ]:
job_usm.write()